In [60]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


In [61]:
import pandas as pd
from xbbg import blp
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime as dt
from datetime import timedelta as td
import statsmodels.api as sm
import re
import matplotlib.image as image
from ipywidgets import interact, IntSlider, Checkbox, Dropdown, Output, HBox, VBox, interactive, interactive_output, ToggleButton,Text, Button, DatePicker, IntText, ToggleButtons, RadioButtons,SelectMultiple, FloatText
from IPython.display import display, clear_output
import itertools
from scipy import stats
from scipy.optimize import minimize 
from scipy.special import ndtr

spot_library = {'eur_6':['EUSA', ' Curncy'],'eur': ['EESWE', ' Curncy'], 'usd': ['USOSFR', ' Curncy'], 
                'gbp': ['BPSWS', ' Curncy'],'chf': ['SFSNT', ' Curncy'],'sek': ['SKSW', ' Curncy'],
                 'nok': ['NKSW', ' Curncy'], 'hkd': ['HDSW', ' Curncy'],'czk': ['CKSW', ' Curncy'],
                 'pln': ['PZSW', ' Curncy'],'ils':['ISSW', ' Curncy'],  'cad':['CDSW', ' Curncy'], 
                 'jpy':['JYSO', ' Curncy'], 'aud': ['ADSW', ' Curncy'],'sgd':['SDSW', ' Curncy'],
                'krw': ['KWSWNI', ' Curncy'],
                'zar': ['SASW', ' Curncy'],
                'nzd': ['NDSW', ' Curncy'],
                'mxn': ['MPSW', ' Curncy']} 

forward_library = {'eur_6': ['EUSA', ' Curncy'], 
                 'eur': ['S0514FS ', ' BLC Curncy'], 
                 'usd': ['S0490FS ', ' BLC Curncy'], 
                 'gbp': ['S0141FS ', ' BLC Curncy'],
                 'chf': ['S0234FS ', ' BLC Curncy'],
                 'sek': ['SD0020FS ', ' BLC Curncy'],
                 'nok': ['SD0313FS ', ' BLC Curncy'],
                 'hkd': ['HDFS', ' Curncy'],
                 'czk': ['S0320FS ', ' BLC Curncy'],
                 'pln': ['S0323FS ', ' BLC Curncy'],
                 'ils': ['ISFS', ' Curncy'],
                 'cad': ['CDFS', ' Curncy'],
                 'jpy': ['S0195FS ', ' BLC Curncy'],
                 'aud': ['SD0302FS ', ' BLC Curncy'],
                'sgd': ['SDFS', ' Curncy'],
                'krw': ['S0205FS ', ' BLC Curncy'],
                'zar': ['SAFS', ' Curncy'],#
                'nzd': ['SD0015FS ', ' BLC Curncy'],
                'mxn': ['SD0083FS ', ' BLC Curncy']} 

################################################# SWAP STRUCTURE BUILDERS ################################################

def cut(some):
    x = 0
    y = some[0]
    while y.isdigit():
        y=some[x]
        x+=1
    return some[:x],some[x:]


def f(tenor):
    x = re.findall(r'\d+',tenor)[0]
    num = ''
    if 'm' in tenor.lower():
        if int(x) // 12 >0:
            num+='1'
        num+=chr(64+(int(x)%12))
    else:
        if len(x) == 1:
            num+='0'
        num +=x
    return num

def t(tenor):
    x = re.findall(r'\d+',tenor)[0]
    num = ''
    if 'm' in tenor.lower():
        if int(x) // 12 >0:
            num+='1'
        else:
            num+='0'
        num+=chr(64+(int(x)%12))
    else:
        if len(x) == 1:
            num+='0'
        num +=x
    return num


def spot_ticker(dex,tenor):
    if dex.lower() == 'mxn':
        y = int(tenor[:-1]) * 13
        num = f'{y//12}{chr(64+(y%12))}'
    else:
        num = tenor[:-1]
    return f'{spot_library[dex][0]}{num}{spot_library[dex][1]}'

def forward_ticker(dex,fwd):
    dex = dex.lower()
    fwd = fwd.lower()
    old = ['eur_6','hkd','ils','cad','sgd','zar']
    
    if cut(fwd)[0] == '0y':
        return spot_ticker(dex,cut(fwd)[1])
    elif dex == 'eur_6':
        F,T = f(cut(fwd)[0]),t(cut(fwd)[1])
        return f'{forward_library[dex][0]}{F}{T}{forward_library[dex][1]}'
    elif dex in old:
        F,T = t(cut(fwd)[0]),t(cut(fwd)[1])
        return f'{forward_library[dex][0]}{F}{T}{forward_library[dex][1]}'
    else:
        
        return f'{forward_library[dex][0]}{fwd.upper()}{forward_library[dex][1]}'


def swap_structure(dex,structure,start, end = 'today',bps = True):
    f_fly = structure.count('/') == 2
    f_crv = structure.count('/') == 1  
    fly = structure.count('s') == 3
    crv = structure.count('s') == 2
    out = max(2 -sum([i.isalpha() for i in structure]),0)
    
    if f_fly or f_crv:
        legs = [forward_ticker(dex,i) for i in structure.split('/')]
    elif fly or crv:
        legs = [spot_ticker(dex,i) for i in [i+'Y' for i in structure.split('s') if i.isdigit()]]
    else:
        legs = forward_ticker(dex,('0Y'*out) + structure)
        
    df = blp.bdh(legs, flds='px_last', start_date=start,end_date=end).fillna(method ='bfill')  *(100 if bps else 1)
    s  = pd.DataFrame({})
    if f_fly or fly:
        x = (2 * df.iloc[:,1]) - (df.iloc[:,0] + df.iloc[:,2])
    elif f_crv or crv:
        x = df.iloc[:,1] - df.iloc[:,0]
    else:
        x = df.iloc[:,0]
        
    s[f'{dex.upper()} {structure}'] = x
    return s

################################################# BOND STRUCTURE BUILDERS ################################################
def bond_ticker(bond_name, bond_tenor):
    cmb = {'ust':1,'dbr':2,'spgb':8,'frtr':4,'btp':5,'ukt':6}
    return f'RV000{cmb[bond_name]}P {bond_tenor.upper()} BLC Curncy'

def bond_structure(bonds,structure, start, end = 'today'):
    if '-' in bonds:
        s = pd.DataFrame({})
        bond1,bond2 = bonds.split('-')
        s[f'{bonds.upper()} {structure}'] = bond_structure(bond1,structure, start, end)[f'{bond1.upper()} {structure}'] - bond_structure(bond2,structure, start, end)[f'{bond2.upper()} {structure}']
        return s
    else:
        if 'y' in structure.lower(): #bond o/r
            tickers = bond_ticker(bonds.lower(), structure)
            base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
            base.columns = [f'{bonds.upper()} {structure}']
            return base
        
        elif structure.lower().count('s') == 2: #bond curve
            tickers = [bond_ticker(bonds.lower(), f'{i}y') for i in structure.lower().split('s') if i != '']
            base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
            base.columns = tickers
            s = pd.DataFrame({})
            s[f'{bonds.upper()} {structure}'] = base[tickers[1]] - base[tickers[0]]
            return s
        
        elif structure.lower().count('s') == 3: # bond fly
            tickers = [bond_ticker(bonds.lower(), f'{i}y') for i in structure.lower().split('s') if i != '']
            base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
            base.columns = tickers
            s = pd.DataFrame({})
            s[f'{bonds.upper()} {structure}'] = (2*base[tickers[1]]) - (base[tickers[0]] + base[tickers[2]])
            return s

################################################# FUTURES STRUCTURE BUILDERS ################################################

def fut_ticker(dex, contract, name = False):
    x  = ''
    if dex.lower()[:3] == 'eur':
        x = 'ER'
    elif dex.lower() == 'usd':
        x = 'SFR'
    elif dex.lower() == 'gbp':
        x = 'SFI'
    
    if name:
        return x
    else:
        return f'{x}{contract} Comdty'   
           
    
def fut_structure(dex,structure, start, end = 'today'):
    name = fut_ticker(dex.lower(), 0,True)
    
    if structure.lower().count('s') == 2: #fut curve
        tickers = [fut_ticker(dex.lower(), int(i)) for i in structure.lower().split('s') if i != '']
        base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
        base.columns = tickers
        s = pd.DataFrame({})
        s[f'{name}{structure.replace("s","")}'] = base[tickers[0]] - base[tickers[1]]
        return s
        
    elif structure.lower().count('s') == 3: # fut fly
        tickers = [fut_ticker(dex.lower(), int(i)) for i in structure.lower().split('s') if i != '']
        base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
        base.columns = tickers
        s = pd.DataFrame({})
        s[f'{name}{structure.replace("s","")}'] = (2*base[tickers[1]]) - (base[tickers[0]] + base[tickers[2]])
        return s
    else:
        tickers = fut_ticker(dex.lower(), structure)
        base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
        base.columns = [f'{name}{structure}']
        return base

################################################# ASW STRUCTURE BUILDERS ################################################

def get_asw(bond_name,tenor,start, end ='today',euro = 'estr'):
    asso_swap = {'ust':'usd','dbr':'eur','spgb':'eur','frtr':'eur','btp':'eur','ukt':'gbp'}
    
    if asso_swap[bond_name] == 'eur' and euro != 'estr':
        tickers = [bond_ticker(bond_name,tenor), spot_ticker(asso_swap[bond_name]+'_6',tenor)]
    else:
        tickers = [bond_ticker(bond_name,tenor), spot_ticker(asso_swap[bond_name],tenor)]
    base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
    base.columns = tickers
    s = pd.DataFrame({})
    s[f'{tenor.upper()} {bond_name.upper()} ASW'] = base[tickers[1]] - base[tickers[0]]
    return s

def asw_structure(bonds,structure, start, end = 'today',euro = 'estr'):
    if '-' in bonds:
        s = pd.DataFrame({})
        bond1,bond2 = bonds.split('-')
        s[f'{bonds.upper()} ASW {structure}'] = asw_structure(bond1,structure, start, end,euro)[f'{bond1.upper()} ASW {structure}'] - asw_structure(bond2,structure, start, end,euro)[f'{bond2.upper()} ASW {structure}']
        return s
    else:
        if 'y' in structure.lower(): #asw o/r
            return get_asw(bonds,structure,start, end,euro)
        
        elif structure.lower().count('s') == 2: #asw curve
            legs = [ f'{i}y' for i in structure.lower().split('s') if i != '']
            short_leg, long_leg = [get_asw(bonds,i,start,end,euro) for i in legs]
            df = short_leg.join(long_leg, how = 'inner')
            s = pd.DataFrame({})
            s[f'{bonds.upper()} ASW {structure}'] = df.iloc[:,1]  - df.iloc[:,0]  
            return s
        
        elif structure.lower().count('s') == 3: # asw fly
            legs = [ f'{i}y' for i in structure.lower().split('s') if i != '']
            short_leg, belly ,long_leg = [get_asw(bonds,i,start,end,euro) for i in legs]
            short_leg = short_leg.join(belly, how = 'inner')
            df = short_leg.join(long_leg, how = 'inner')
            s = pd.DataFrame({})
            s[f'{bonds.upper()} ASW {structure}'] = (2 * df.iloc[:,1])  - (df.iloc[:,0] + df.iloc[:,2])  
            return s
        

######################################### TICKER HANDLERS #################################################################

def get_other(ticker,attribute,title,start,end='today' ):
    if 'price' in attribute.lower():
        attribute = 'px_last'
    else:
        attribute = 'YLD_YTM_MID'
    base = blp.bdh(ticker,attribute,start,end)
    base.columns = [title]
    return base

######################################### INVOICE SPREAD STRUCTURE BUILDERS ################################################


def get_ivsp(country,start, end ='today',which = 'TUA',euro ='estr'):
    tkr = {'USD':['TUAISPS','3YAISPS','FVAISPS','TYAISPS','UXYAISPS','USAISPS','WNAISPS'],
           'GER':['DUAISP','OEAISP','RXAISP','UBAISP'],
           'ITA':['BTAISP','IKAISP'],
           'FRA':['OATAISP'],
           'GBP':['GAISPO']}
    
    if country.upper() == 'USD' or country.upper() == 'GER' or country.upper() == 'ITA' :
        for i in tkr[country.upper()]:
            if which in i:
                future = i
    else:
        future = tkr[country.upper()][0]
    
    if country.lower() in ['ger','ita','fra'] and euro == 'estr':
        future += 'E'
    final_ticker = f'{future} Comdty'
    base = blp.bdh(final_ticker,'px_last',start,end)
    base.columns = [f'{future} Comdty']    
    return base
                
######################################## MEETING DATES HANDLER ##################################################################
    
def md_swap_ticker(bank, meeting,m_of_m =False):
    central_bank = {'FOMC':['USSOFED',' Curncy'],
                'ECB':['EESF','A Curncy'],
                'MPC':['GPSF','A Curncy']}
    ticker = f'{central_bank[bank.upper()][0]}{meeting}{central_bank[bank.upper()][1]}' 
    if m_of_m:
        return blp.bdp(ticker, 'SW_EFF_DT').values.flatten()[0].strftime('%b')
    else:
        return ticker
    
def get_md_swap(bank,structure,start,end  = 'today'):
    if structure.lower().count('s') == 2:
        tickers = [md_swap_ticker(bank.upper(),int(i)) for i in structure.lower().split('s') if i != ''] 
        lbls = ''.join([md_swap_ticker(bank.upper(),int(i), True) for i in structure.lower().split('s') if i != ''])
        base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
        base.columns = tickers
        s = pd.DataFrame({})
        s[f'{lbls} {bank.upper()}'] = base[tickers[1]] - base[tickers[0]]
        return s
        
    if structure.lower().count('s') == 3:
        tickers = [md_swap_ticker(bank.upper(),int(i)) for i in structure.lower().split('s') if i != '']
        lbls = ''.join([md_swap_ticker(bank.upper(),int(i), True) for i in structure.lower().split('s') if i != ''])            
        base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
        base.columns = tickers
        s = pd.DataFrame({})
        s[f'{lbls} {bank.upper()}'] = (2*base[tickers[1]] )- (base[tickers[0]] + base[tickers[2]])
        return s
    else:
        tickers = md_swap_ticker(bank.upper(), structure)
        base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
        base.columns = [f'{md_swap_ticker(bank.upper(), structure,True)}{bank.upper()}']
        return base

    
################################################# FX HANDLER ############################################################
def get_fx_pair(pair,start,end='today'):
    ticker = f'{pair.upper()} Curncy'
    base = blp.bdh(ticker,'px_last',start,end)
    base.columns = [pair.upper()]
    return base

######################################## BOND FUTURE HANDLER ##################################################################
def get_bondf_ticker(name):
    tic_dic = {'DU':'DUA Comdty','OE':'OEA Comdty','RX': 'RXA Comdty','UB': 'UBA Comdty',
                'BTS': 'BTSA Comdty','IK': 'IKA Comdty',
                'OAT': 'OATA Comdty',
                'G':'G A Comdty',
                'TU': 'TUA Comdty','FV': 'FVA Comdty', 'TY': 'TYA Comdty','WN': 'WNA Comdty','UXY': 'UXYA Comdty'}
    return tic_dic[name.upper()]
    

def get_bond_fut(_str, start,end = 'today', fld = 'yield'):
    fld = 'YLD_YTM_MID' if fld.lower() == 'yield' else 'PX_LAST' 
    _str = _str.upper()
    label = _str.split('/')
    legs = [get_bondf_ticker(i) for i in label]
    bps = 100 if fld == 'YLD_YTM_MID' else 1
    base  = blp.bdh(legs,fld,start,end) * bps
    base.columns = label
    s = pd.DataFrame({})
    
    if _str.count('/') == 2:
        s[_str] = 2 * base[label[1]] - (base[label[0]] + base[label[2]]) 
            
    elif _str.count('/') == 1:
        s[_str] =  base[label[1]] - base[label[0]] 
                                        
    else:
        s = base
        
    return s
    

######################################## INFL SWAP HANDLER ##################################################################



######################################## MASTER HANDLER ##################################################################


def function_for_bb(trades,_from,_to):
    master_c = pd.DataFrame({})
    for i in trades.keys():
        if len(trades[i]) != 0:
            for j in trades[i]:
                if i == 'SWAP':
                    h = swap_structure(dex = j[0].lower() ,structure=j[1].lower() ,start = _from , end = _to)                                
                elif i == 'FUTURE':
                    h = fut_structure(dex = j[0].lower() ,structure=j[1].lower() ,start = _from , end = _to)                              
                elif i == 'BOND':
                    h = bond_structure(bonds = j[0].lower() ,structure=j[1].lower() ,start = _from , end = _to)
                elif i ==  'ASW':
                    h = asw_structure(bonds = j[0].lower() ,structure=j[1].lower() ,start = _from , end = _to, euro = j[2].lower())
                elif i == 'XMKT SWAP':
                    h1 = swap_structure(dex = j[0].lower() ,structure=j[1].lower() ,start = _from , end = _to)
                    h2 = swap_structure(dex = j[2].lower() ,structure=j[1].lower() ,start = _from , end = _to)
                    h_int = h1.join(h2, how = 'inner')
                    h = pd.DataFrame(h_int.iloc[:,0] - h_int.iloc[:,1], columns = [f'{j[0]}-{j[2]} {j[1]}'])
                elif i == 'MD SWAP':
                    h = get_md_swap(bank = j[0].upper(),structure=j[1].lower() ,start = _from , end = _to)
                elif i ==  'BOND FUTURE':
                    h = get_bond_fut(_str = j[0],start = _from , end = _to,fld =  j[1])
                elif i == 'FX':
                    h = get_fx_pair(pair = j[0].upper(),start = _from , end = _to)
                elif i == 'OTHER/CIX':
                    h = get_other(ticker = j[0], attribute= j[1], title = j[2], start = _from , end = _to)            

                if master_c.shape == (0,0):
                    master_c = h
                else:
                    master_c = master_c.join(h, how = 'inner')
    return master_c
                            
                            
    

In [62]:

clrs = ['whites','reds','greens','blues']
def fut_presets(dex,clr):
    color = {key : range(1+(clrs.index(key)*4),5+(clrs.index(key)*4)) for key in clrs}
    fin_clr = []
    for i in clr.lower().split(', '):
        fin_clr += list(color[i])
    for i in fin_clr:
        yield (dex.upper(), str(i))

clr_dd = [', '.join([m.capitalize() for m in j]) for i in range(1,len(clrs)+1) for j in itertools.combinations(clrs, i) ]


sp_or_fw = ['Spot Tenors', 'Forward Gaps']
yc = ['(up to 10y)','(up to 15y)','(up to 30y)','(up to 50y)']
srt = ['Spot Curves', 'Spot Flys', 'Forward Curves']
swap_ops = [f'{i} {j}' for i in sp_or_fw for j in yc] + srt
bond_ops = ['Outright', 'Curves', 'Flys']

def swap_presets(dex, build):
    
    spot_front = [f'{i}y' for i in range (1,11)]
    spot_mid = spot_front + ['12y','15y']
    spot_back = spot_mid + ['20y','25y','30y']
    spot_ultraback =spot_back + ['40y','50y']
    
    fwds_front = [f'{i}y1y' for i in range (1,10)]
    fwds_mid = fwds_front +['10y2y','12y3y']
    fwds_back = fwds_mid + ['15y5y','20y5y','25y5y']
    fwds_ultraback = fwds_back + ['30y10y','40y10y']
    
    spot_curves = ['1s2s','2s5s','2s10s','5s10s','5s30s','10s30s']
    spot_flys = ['1s2s3s','1s3s5s', '2s5s10s', '5s10s30s']
    fwds_curves = ['1y1y/5y5y','1y2y/1y10y']
    answer = []
    if 'spot' in build.lower():
        if '10y' in build.lower():
            answer = spot_front
        elif '15y' in build.lower():
            answer = spot_mid
        elif '30y' in build.lower():
            answer = spot_back
        elif '50y' in build.lower():
            answer = spot_ultraback
        elif 'curves' in build.lower():
            answer = spot_curves
        elif 'flys' in build.lower():
            answer = spot_flys 
        else:
            answer = spot_curves + spot_flys
    else:
        if '10y' in build.lower():
            answer = fwds_front
        elif '15y' in build.lower():
            answer = fwds_mid
        elif '30y' in build.lower():
            answer = fwds_back
        elif '50y' in build.lower():
            answer = fwds_ultraback
        else:
            answer = fwds_curves
    for i in answer:
        yield(dex.upper(),i)
        
def bond_spline_presets(bond,build):
    answer = {'Outright': ['2y','5y','10y','30y'],
             'Curves': ['2s5s','2s10s','5s10s','5s30s','10s30s'], 
              'Flys':['2s5s10s', '5s10s30s']}
    for i in answer[build]:
        yield (bond.upper(), i)
        
def asw_spline_presets(bond,build,euro = 'ESTR'):
    answer = {'Outright': ['2y','5y','10y','30y'],
             'Curves': ['2s5s','2s10s','5s10s','5s30s','10s30s'], 
              'Flys':['2s5s10s', '5s10s30s']}
    for i in answer[build]:
        yield (bond.upper(), i,euro)

def custom_presets(broker):
    if broker == "MR Presets":
        return {'SWAP': [('USD','2y1y/3y2y/5y2y'),('GBP','2y1y/3y2y/5y2y'),('EUR','2y1y/3y2y/5y2y'),('EUR_6','2y1y/3y2y/5y2y'),
                         ('USD','5y2y/7y3y'),('GBP','5y2y/7y3y'),('EUR','5y2y/7y3y'),('EUR_6','5y2y/7y3y'),
                         ('USD','5y5y/10y5y'),('GBP','5y5y/10y5y'),('EUR','5y5y/10y5y'),('EUR_6','5y5y/10y5y'),
                         ('USD','5y5y/10y5y/15y5y'),('GBP','5y5y/10y5y/15y5y'),('EUR','5y5y/10y5y/15y5y'),('EUR_6','5y5y/10y5y/15y5y'),
                         ('USD','10y10y/20y10y'),('GBP','10y10y/20y10y'),('EUR','10y10y/20y10y'),('EUR_6','10y10y/20y10y'),
                         ('USD','20y10y/30y20y'),('GBP','20y10y/30y20y'),('EUR','20y10y/30y20y'),('EUR_6','20y10y/30y20y'),
                         ('USD','3y2y/5y2y/7y3y'),('GBP','3y2y/5y2y/7y3y'),('EUR','3y2y/5y2y/7y3y'),('EUR_6','3y2y/5y2y/7y3y')
                        ]}
        

In [63]:
trades = {'SWAP':[],'FUTURE':[],'BOND':[], 'BOND FUTURE': [],'ASW':[],'MD SWAP': [] ,'XMKT SWAP':[],'FX':[] ,'OTHER/CIX': []}
master = pd.DataFrame({})


In [64]:

x = ['-'] + [i for i in trades.keys()]
assetW = Dropdown(options = x,description = 'Instrument')

output1 = Output()
output2 = Output()
outputn = Output()

def build_data_list(asset):

    eugov = ['DBR','SPGB','FRTR','BTP','BTP-DBR','FRTR-DBR','SPGB-DBR','BTP-SPGB','FRTR-SPGB']

    
    style = {'description_width': 'initial'}
    b_or_i = {'-': Text(value = 'Choose Instrument',disabled = True),
        'SWAP':Dropdown(options = [i.upper() for i in spot_library.keys()], description = 'Currency'),
         'FUTURE':Dropdown(options = ['USD','EUR','GBP'], description = 'Currency'),
         'BOND':Dropdown(options = ['UST','DBR','SPGB','FRTR','BTP','UKT','BTP-DBR','FRTR-DBR','SPGB-DBR','BTP-SPGB','FRTR-SPGB'], description = 'Bond'),
         'BOND FUTURE': Dropdown(options = ['Yield', 'Price'], description = 'Field'),
        'MD SWAP': Dropdown(options = ['FOMC','ECB','MPC'], description = 'Central Bank'),
         'ASW':Dropdown(options = ['UST','DBR','SPGB','FRTR','BTP','UKT','BTP-DBR','FRTR-DBR','SPGB-DBR','BTP-SPGB','FRTR-SPGB'], description = 'Bond'),
              'FX': Text(description = 'Pair'),
              'XMKT SWAP':Dropdown(options = [i.upper() for i in spot_library.keys()], description = 'Currency'),
              'OTHER/CIX':Text(description = 'Ticker')}
    
    what_trade = {'-': Text(value = 'Choose Instrument',disabled = True),
                  'SWAP':Text(description = 'Structure'),
         'FUTURE':Text(description = 'Structure'),
         'BOND':Text(description = 'Structure'),
        'BOND FUTURE':Text(description = 'Structure'),
         'ASW':Text(description = 'Structure'),
        'MD SWAP': Text(description = 'Structure'),
        'XMKT SWAP':Text(description = 'Structure'),
                  'FX':Text(value = ' ',disabled = True),
        'OTHER/CIX':Dropdown(options = ['','Last Price','YTM'],description = 'Attribute')}
    
    
    what_trade_p = {'-': Text(value = 'Choose Instrument',disabled = True),
                  'SWAP':Dropdown(options = ['-']+swap_ops, description = 'Preset'),
         'FUTURE':Dropdown(options = ['-']+clr_dd, description = 'Preset'),
         'BOND':Dropdown(options = ['-']+bond_ops, description = 'Preset'),
        'BOND FUTURE':Text(value = '-',disabled = True),
         'ASW':Dropdown(options = ['-']+bond_ops, description = 'Preset'),
             'MD SWAP':Text(value = '-',disabled = True),
             'XMKT SWAP':Text(value = '-',disabled = True),
             'FX': Text(value = '-',disabled = True),
             'OTHER/CIX':Dropdown(options = ['-',"MR Presets"], description = 'Preset')}
    
    final = {'-': Text(value = 'Choose Instrument',disabled = True),
                  'SWAP':Text(value = '-',disabled = True),
         'FUTURE':Text(value = '-',disabled = True),
         'BOND':Text(value = '-',disabled = True),
        'BOND FUTURE':Text(value = '-',disabled = True),
         'ASW':Dropdown(options = ['ESTR','EURIBOR'] , description = 'Against (if EUGOV)',style=style),
             'MD SWAP':Text(value = '-',disabled = True),
             'XMKT SWAP':Dropdown(options = [i.upper() for i in spot_library.keys()], description = 'Against'),
             'FX': Text(value = '-',disabled = True),
             'OTHER/CIX':Text(description = 'Label')}
    
    include_in_list = Button(description = 'Add Structure')
    removefrom_list = Button(description = 'Remove Last')
    include__preset = Button(description = 'Add Preset')
    
    global trades
    
    
    def on_button_clicked1(b):
        with output1:
            if asset =='-' or what_trade[asset].value == '':
                pass
            else:
                if asset in ['XMKT SWAP','OTHER/CIX','ASW']:
                    trades[asset].append((b_or_i[asset].value,what_trade[asset].value,final[asset].value))
                elif asset == 'BOND FUTURE':
                    trades[asset].append((what_trade[asset].value ,b_or_i[asset].value))
                else:
                    trades[asset].append((b_or_i[asset].value,what_trade[asset].value))
                    
                assetW.value = '-'


    def on_button_clickedn(b):
        with outputn:
            if asset =='-' or what_trade_p[asset].value == '-':
                pass
            else:
                if asset == 'FUTURE':
                    for tup in fut_presets(b_or_i[asset].value,what_trade_p[asset].value):
                        trades[asset].append(tup)
                elif asset == 'SWAP':
                    for tup in swap_presets(b_or_i[asset].value,what_trade_p[asset].value):
                        trades[asset].append(tup)
                elif asset == 'BOND':
                    for tup in bond_spline_presets(b_or_i[asset].value,what_trade_p[asset].value):
                        trades[asset].append(tup)
                elif asset == 'ASW':
                    for tup in asw_spline_presets(b_or_i[asset].value,what_trade_p[asset].value,final[asset].value):
                        trades[asset].append(tup)
                        
                elif asset == 'OTHER/CIX':
                    custom_trades = custom_presets(what_trade_p[asset].value) 
                    for a in custom_trades.keys():
                        trades[a] += custom_trades[a]
                    
                assetW.value = '-'    
                                     
                
    def on_button_clicked2(b):
        with output2:
            if assetW == '-':
                pass
            else:
                trades[asset].pop()
                assetW.value = '-'
    
    
    
    display(HBox([b_or_i[asset],what_trade[asset],final[asset]]))
    display(what_trade_p[asset])
        
        
    include_in_list.on_click(on_button_clicked1)
    removefrom_list.on_click(on_button_clicked2)
    include__preset.on_click(on_button_clickedn)
    display(HBox([include_in_list,removefrom_list,include__preset]))
    
    max_len = max([len(trades[i]) for i in trades.keys()])
    y = {}
    for i in trades.keys():
        if i == 'XMKT SWAP':
            y[i] = [f'{i[0]}-{i[2]} {i[1]}' for i in trades[i]] + [float('nan')]*(max_len - len(trades[i]))
            
        elif i  =='ASW':# and i[0] in eugov:
            
            y[i] = [f'{i[0]}-{i[1]} v {i[2]}'if i[0] in eugov else f'{i[0]} {i[1]}' for i in trades[i] ] + [float('nan')]*(max_len - len(trades[i]))
        
        elif i == 'BOND FUTURE':
            y[i] = [j[0] for j in trades[i]] + [float('nan')]*(max_len - len(trades[i]))
            
            
        elif i == 'OTHER/CIX':
            y[i] = [f'{i[2]}' for i in trades[i]] + [float('nan')]*(max_len - len(trades[i]))
            
        else:
            y[i] = [f'{i[0]} {i[1]}' for i in trades[i]] + [float('nan')]*(max_len - len(trades[i]))
    
    display(pd.DataFrame(y).style.format('{}',na_rep = ''))
    
interactive(build_data_list,asset=assetW)


interactive(children=(Dropdown(description='Instrument', options=('-', 'SWAP', 'FUTURE', 'BOND', 'BOND FUTURE'…

In [65]:
output3 = Output()
get_data_frombb = Button(description = 'Pull BBG Data')
gen_from = DatePicker(value =dt(dt.today().year - 1, dt.today().month,dt.today().day),description = 'Start')
gen_to = DatePicker(value = dt.today(),description = 'End')
display(HBox([gen_from,gen_to]))
display(get_data_frombb,output3)

def on_button_clicked3(b):
    global master
    with output3:
        clear_output()
        master = function_for_bb(trades, gen_from.value,gen_to.value)
        dis = pd.DataFrame({},index = ['CURRENT', 'AVG MOVE', 'DAILY VOL'])
        dis.loc['CURRENT',master.columns] = master.loc[master.index[-1]]
        dis.loc['AVG MOVE',master.columns] = (master.diff().mean()) 
        dis.loc['DAILY VOL',master.columns] = (master.diff().std())

        display(dis.style.format('{:.2f}',na_rep = ''))
get_data_frombb.on_click(on_button_clicked3)    

Button(description='Pull BBG Data', style=ButtonStyle())

Output()

# MEAN REVERSION MODELLING


 ### MODEL DIAGNOSTICS

In [66]:
## MODEL FITTING FUNCTIONS

def adf_test(trade):
    lengths = [30,60,90,183,365,730]
    l_label = ['1M','2M','3M','6M','1Y','2Y']
    dfoutputs = []
    for l in lengths:
        if trade.shape[0] >= l:
            test = sm.tsa.adfuller(trade.loc[trade.index[-l]:], autolag="BIC")
            op = pd.Series(test[0:4],index=[ "Test Statistic", "p-value","Lags Used","Observations"])
            for key, value in test[4].items():
                op[f"Critical Value ({key})"] = value
            dfoutputs.append(op)
    
    fs_test = sm.tsa.adfuller(trade, autolag="BIC")
    fs_op = pd.Series(fs_test[0:4],index=[ "Test Statistic", "p-value","Lags Used","Observations"],)
    for key, value in fs_test[4].items():
        fs_op[f"Critical Value ({key})"] = value
    dfoutputs.append(fs_op)
    
    dfoutput = pd.concat(dfoutputs,axis=1) 
    dfoutput.columns =  l_label[:max(len(dfoutputs) -1,0 )] + ['Full Sample']
    
    to_display = dfoutput.T.style.format({'Lags Used':'{:.0f}',
                                          'Observations':'{:.0f}',
                                          'Test Statistic':'{:.2f}',
                                          'p-value':'{:.2f}',
                                          'Critical Value (1%)':'{:.2f}',
                                          'Critical Value (5%)':'{:.2f}',
                                          'Critical Value (10%)':'{:.2f}'})
    return to_display


def kpss_test(trade):
    warnings.simplefilter("ignore")
    lengths = [30,60,90,183,365,730]
    l_label = ['1M','2M','3M','6M','1Y','2Y']
    kpss_outputs = []
    for l in lengths:
        if trade.shape[0] >= l:
            test = sm.tsa.stattools.kpss(trade.loc[trade.index[-l]:], regression="c",nlags='auto')
            op = pd.Series(test[0:3],index=[ "Test Statistic", "p-value","Lags Used"])
            for key, value in test[3].items():
                op[f"Critical Value ({key})"] = value    
            kpss_outputs.append(op)
    
    fs_test = sm.tsa.stattools.kpss(trade, regression="c",nlags = 'auto')
    fs_op = pd.Series(fs_test[0:3],index=[ "Test Statistic", "p-value","Lags Used"])
    for key, value in fs_test[3].items():
        fs_op[f"Critical Value ({key})"] = value
    kpss_outputs.append(fs_op)
    
    kpss_output = pd.concat(kpss_outputs,axis=1)
    kpss_output.columns = l_label[:max(len(kpss_outputs) -1,0 )] + ['Full Sample']
    
    to_display = kpss_output.T.style.format({'Lags Used':'{:.0f}',
                                          'Test Statistic':'{:.2f}',
                                          'p-value':'{:.2f}',
                                          'Critical Value (1%)':'{:.2f}',
                                          'Critical Value (5%)':'{:.2f}',
                                             'Critical Value (2.5%)':'{:.2f}',
                                          'Critical Value (10%)':'{:.2f}'})
    return to_display
    

    
def bucketing(data, buckets = 5, kind = 'm', low = 0.1,high = 0.9):
    date = data.index
    change = data.diff(1).shift(-1).dropna()
    data = data.drop(axis = 0, index = data.index[-1])
    lo= [data[data<=data.quantile(low)]]
    hi = [data[data>data.quantile(high)]]
    trim = data[(data.quantile(low)<data)&(data<=data.quantile(high))]
    a = [trim.quantile(i/100) for i in range(0,101,100//(buckets))]
    a.pop(0),a.pop(-1)
    b = lo+[trim[(a[i]<trim)&(trim<a[i+1])] for i in range(len(a)-1)]+hi
    c = [data.quantile(0)] + a + [data.quantile(1)]
    mid = [round((c[i+1] + c[i])/2,2) for i in range(len(c)-1)]
    chg = [change[i.index].mean() for i in b]
    std = [change[i.index].std() for i in b]
    
    
    return pd.DataFrame({'MID':mid,'CHANGE': chg, 'VOL':std})
    
def data_prep(data):
    x_t = data.values
    xt_1 = data.shift(1).dropna().values
    tau = np.ones_like(xt_1)
    return x_t, xt_1,tau


def MLE_Norm(parameters,dataset):
    x_t, x_t_1, tau = data_prep(dataset)
    kappa,mu,sigma = parameters
    mean = mu + ((x_t_1 - mu) * np.exp(-kappa * tau))
    std_dev = sigma * np.sqrt(((1-np.exp(-2*kappa*tau)))/(2*kappa))
    mean = np.concatenate(([mu],mean))
    std_dev = np.concatenate(([sigma/np.sqrt(2*kappa)],std_dev))
    LL = np.sum(stats.norm.logpdf(x_t, mean, std_dev))
    neg_LL = -1*LL
    return neg_LL


def simulate_OU(X0,params,paths=1e6, T=90):
    
    kappa,mu,sigma = params
    delta_t = 1     
    
    X = np.zeros((paths,T))
    X[:,0] = X0
    W = stats.norm.rvs( loc=0, scale=1, size=(paths,T-1) )
    
    #Uncomment for Euler Maruyama
    #for t in range(0,N-1):
    #X[:,t+1] = X[:,t] + kappa*(mu - X[:,t])*dt + sigma * np.sqrt(dt) * W[:,t]

    std_dt = np.sqrt((sigma**2) * (1-np.exp(-2*kappa*delta_t)) /(2*kappa) )
    for t in range(0,T-1):
        X[:,t+1] = mu + np.exp(-kappa*delta_t)*(X[:,t]-mu) + std_dt * W[:,t]
        
    return X

    
def OU_mean(params,x_c = None,days = 10):
    kappa,mu,sigma = params
    if x_c != None:
        mu += (x_c-mu)*np.exp(-kappa*days) 
    return mu
        
        
def OU_var(params,x_c = None,days = 10):
    kappa,mu,sigma = params
    var = (sigma ** 2)/(2* kappa)
    if x_c != None:
        var *= (1-np.exp(-2*kappa*days))
    return var 

def density(x,params,x_c = None,days = 10, kind = 'cdf'):
    mean = OU_mean(params, x_c, days)
    std = np.sqrt(OU_var(params, x_c, days) )   
    
    if kind == 'cdf':
        return stats.norm.cdf(x,mean,std)
    else:
        return stats.norm.pdf(x,mean,std)

def half_life(kappa):
    return (np.log(2)/kappa)   
    

def hitting_time(sim_df,current_level,target,stop,direction):
    
    pay_check = (direction.upper() == 'PAY') & (stop < current_level) & (current_level < target)
    rec_check = (direction.upper() == 'REC') & (stop > current_level) & (current_level > target)
    
    up_days = []
    up_count = 0
    down_days = []
    down_count = 0
    
    if not pay_check and not rec_check:
        return 'Please check direction, stop and target.'
    
    else:    
        up, down = max(target,stop), min(target,stop)
        for path in range(sim_df.shape[0]):
            for t in range(1,sim_df.shape[1]):
            
                if sim_df[path,t] >= up:
                    up_days.append(t+1)
                    up_count += 1
                    break
                    
                elif sim_df[path,t] <= down:
                    down_days.append(t+1)
                    down_count += 1
                    break

                    
        
        tp_days = up_days if pay_check else down_days
        sl_days = down_days if pay_check else up_days
        tp_pct =  100 * (up_count if pay_check else down_count) / sim_df.shape[0]
        sl_pct = 100 * (down_count if pay_check else up_count) / sim_df.shape[0]
        
        result = pd.DataFrame({ 'Target Reached':[f"{round(tp_pct,1)}%", f"{round(sum(tp_days)/len(tp_days),1)} days"],
                               'Stop Triggered':[f"{round(sl_pct,1)}%", f"{round(sum(sl_days)/len(sl_days),1)} days"]
                             }, index = ['Proportion','Average days'])
        return result
    
    
        

In [67]:
model_ready = False
model_metrics = None 
time_series  = None

WHICH_TRADE_W = Dropdown(options = master.columns, description = 'Trade')
LOOKBACK3_W = IntText(value = 30, description = 'Lookback')
ANALYTICS_W = ToggleButtons(options = ['ADF Test','KPSS Test','Bucket Plot'], description = 'Show Model')

output6 = Output()

ANALYSE_B = Button(description = 'Run Analysis')
FIT_MODEL_B = Button(description = 'Fit Model')
UPDATE1_B = Button(description = 'Update Dropdown')

display(VBox([WHICH_TRADE_W, LOOKBACK3_W,ANALYTICS_W]))
display(output6,VBox([ANALYSE_B,FIT_MODEL_B, UPDATE1_B]))

def on_button_clicked6(b):
    with output6:
        clear_output()
        subset =  master.copy().loc[master.index[-LOOKBACK3_W.value]:,WHICH_TRADE_W.value]
        diagnostic_plot = bucketing(subset)
        #display(subset.describe()[['mean','std']])
        if ANALYTICS_W.value == 'ADF Test':
            print('ADF test is used to check for stationarity.\nLow p-value => more chance of stationarity')
            print('Stationary structures make good candidates for reversion modelling')
            display(adf_test(subset))
        
        elif ANALYTICS_W.value == 'KPSS Test':
            print('KPSS test is used to check for a certain kind of stationarity, namely trend-stationarity.\nHigh p-value => more chance of stationarity')
            print('Stationary structures make good candidates for reversion modelling')
            display(kpss_test(subset))
        
        elif ANALYTICS_W.value == 'Bucket Plot':
            fig,(ax1,ax2) = plt.subplots(nrows=1,ncols=2, figsize = (15,4))
            plt.setp((ax1,ax2), xticks=range(diagnostic_plot.shape[0]), xticklabels=diagnostic_plot['MID'])

            ax1.plot(diagnostic_plot['CHANGE'],color = 'b')
            ax1.set_title('AVERAGE MOVE')
            ax1.set_ylabel('MOVE')
            ax1.set_xlabel('Bucket Level')

            ax2.plot(diagnostic_plot['VOL'],color = 'r')
            ax2.set_title('AVERAGE VOL')
            ax2.set_ylabel('VOLATILITY')
            ax2.set_xlabel('Bucket Level')

            plt.show()  
            
        else:
            pass

def on_button_clicked7(b):
    global model_ready
    global model_metrics
    global time_series
    with output6:
        clear_output()
        subset =  master.copy().loc[master.index[-LOOKBACK3_W.value]:,WHICH_TRADE_W.value]
        initial_estimates = [0.5,subset.describe()['mean'],subset.describe()['std']]
        mle_model = minimize(MLE_Norm,initial_estimates, method = 'Nelder-Mead',args = (subset))
        model_ready = True
        time_series = subset
        col_val = f'{WHICH_TRADE_W.value}'
        model_metrics = pd.DataFrame({},columns = [col_val], index = ['Current Level',
                                         'Speed of Mean Reversion',
                                         'Long Run Mean',
                                         'Volatility',
                                         'Half-Life'])

        model_metrics.at['Current Level',col_val] = subset[subset.index[-1]]
        model_metrics.at['Speed of Mean Reversion',col_val] = mle_model['x'][0]
        model_metrics.at['Long Run Mean',col_val] = mle_model['x'][1]
        model_metrics.at['Volatility',col_val] = mle_model['x'][2]
        model_metrics.at['Half-Life',col_val] = half_life(mle_model['x'][0])
        display(mle_model)
        
        

def on_button_clicked10(b):
    with output6:
        clear_output()
        WHICH_TRADE_W.options = master.columns
        
    
            
ANALYSE_B.on_click(on_button_clicked6)
FIT_MODEL_B.on_click(on_button_clicked7)
UPDATE1_B.on_click(on_button_clicked10)

Output()

 ### MODEL FORECASTING

In [68]:
sim_result = None
sim_ready = False

In [69]:


DIRECTION_W  = RadioButtons(options = ['PAY', 'REC'], description = 'Direction')
TARGET_W = FloatText(description = 'Enter Target')
STOP_W = FloatText(description = 'Enter Stop')
ANALYTICS2_W = ToggleButtons(options = ['Metrics','Chart','Transition Densities','Simulation Result'], description = 'Study')

output7 = Output()

ANALYSE2_B = Button(description = 'Show Study')
RUN_SIM_B  = Button(description = 'Run Simulations')

display(VBox([HBox([TARGET_W, STOP_W]),DIRECTION_W,ANALYTICS2_W]))
display(output7,VBox([ANALYSE2_B, RUN_SIM_B]))


def on_button_clicked8(b):
    with output7:
        clear_output()
        if model_ready:
            current_level = time_series[time_series.index[-1]]
            params = model_metrics.iloc[1:4].values.flatten()
            forecast_length =min(time_series.shape[0],90)
            forecast_period = pd.date_range(time_series.index[-1],time_series.index[-1] + td(forecast_length) )
            forecasts = pd.DataFrame({}, index = forecast_period, columns = ['Lower 2 Sigma Band', 'Expected Path', 'Upper 2 Sigma Band'])
            forecasts.loc[time_series.index[-1]] = current_level
            for delta_t, h in enumerate(forecast_period):
                if h != 0:
                    mean = OU_mean(params,x_c=time_series[time_series.index[-1]], days = delta_t)
                    std = np.sqrt(OU_var(params,x_c=time_series[time_series.index[-1]], days = delta_t))
                    forecasts.at[h,'Lower 2 Sigma Band'] = mean - 1.96 * std 
                    forecasts.at[h,'Expected Path'] = mean
                    forecasts.at[h,'Upper 2 Sigma Band'] = mean + 1.96 * std
            
            if ANALYTICS2_W.value == 'Metrics':
                display(model_metrics)
            
            elif ANALYTICS2_W.value == 'Transition Densities':
                
                
                
                
                Z = np.linspace(-4,4,1000)
                fig, ax = plt.subplots(figsize = (15,8))
                
                for i in range(1,5):
                    mean, std = OU_mean(params, x_c = current_level, days = i), np.sqrt(OU_var(params, x_c = current_level, days = i))
                    x = np.array([mean + (std * z) for z in Z]) 
                    ax.plot(x,density(x,x_c = current_level ,days = i , kind = 'pdf', params = params),label = f'{i}D')
                    
                
                mean, std = OU_mean(params, x_c = None), np.sqrt(OU_var(params, x_c = None))
                x = np.array([mean + std * z for z in Z])
                ax.plot(x,density(x, kind = 'pdf',params = params),label = 'UNCONDITIONAL', color = 'darkblue')                
                shaded_region_1,shaded_region_2  =  (x <= mean - 1.282 * std), (x >= mean + 1.282 * std)
                
                ax.fill_between(x, density(x,x_c = None , kind = 'pdf', params = params), where=shaded_region_1, alpha = 0.1,color = 'darkblue')
                ax.fill_between(x, density(x,x_c = None , kind = 'pdf', params = params), where=shaded_region_2, alpha = 0.1,color = 'darkblue')
                ax.axvline(current_level, color ='k', linestyle = '--', linewidth = 0.8,label = 'Current Level')
                ax. set_title(f'Transition densities: {model_metrics.columns[0]}')    
                ax.set_ylim(bottom = 0)
                ax.legend()
                plt.show()
                
            elif ANALYTICS2_W.value == 'Chart':
                fig,ax = plt.subplots(figsize = (15,8))
                ax.plot(time_series)
                for i in forecasts.columns:
                    ax.plot(forecasts[i], label = i)
            
                
                ax.axhline(params[1],color = 'k', linestyle = '--')
                ax.set_title(model_metrics.columns[0])
                ax.legend()
                plt.show()
                
            elif ANALYTICS2_W.value == 'Simulation Result':
                if sim_ready:
                    stop = STOP_W.value
                    target = TARGET_W.value
                    result = hitting_time(sim_result,current_level,target,stop,DIRECTION_W.value)
                    display(result)
                    
                else:
                    print('Please run simulation.')
                
        else:
            print('Please fit a model in the section above')
            
def on_button_clicked9(b):
    global sim_result
    global sim_ready
    with output7:
        clear_output()
        if model_ready:
            current_level = time_series[time_series.index[-1]]
            params = model_metrics.iloc[1:4].values.flatten()
            paths = int(1e6)
            T = time_series.shape[0]
            print('Running Simulations...')
            sim_ready = True
            sim_result = simulate_OU(X0 = current_level, params=params, paths = paths, T = T)
            clear_output()
            print('Simulation Complete')
            
        else:
            print('Please fit a model in the section above')
            
        
        
    

ANALYSE2_B.on_click(on_button_clicked8)
RUN_SIM_B.on_click(on_button_clicked9)

Output()